In [ ]:
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd
from numpy import math

import covid_utils

In [ ]:
utils = covid_utils.Utils()

df = utils.read_counties_csv()
state_names = df["state"].unique()

df = df.drop(columns=["county", "state", "fips"])
state_names

In [ ]:
last_updated = df["date"].max()
print("Last updated:", last_updated)
print("Time since first case:", df["date"].max() - df["date"].min())

In [ ]:
today = df.loc[df["date"] == last_updated]

lo, hi = utils.get_top(today, "deaths", num_to_show=10)
print(hi)

In [ ]:
counties = utils.compute_diffs(df, window_size=5)
counties

In [ ]:
plt.rcParams['figure.figsize'] = [15, 10]
utils.generate_plots(counties, ['New York'])

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
utils.plot_cases_by_weekday(counties, "New York City, New York")